In [ ]:
import uproot
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
import matplotlib as mpl

from util import *
import var
import cut
import data
import hist

import importlib

from pyanalib import panda_helpers

In [ ]:
mpl.rc('font', size=14)

dosave = True
savedir = "./plots_8_1_23/"

import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
MASS = 300

In [ ]:
f = "/icarus/data/users/gputnam/dimuon-data/higgs_M%i_DMCP2023G_trk.df" % MASS

In [ ]:
df = pd.read_hdf(f, key="trk")

In [ ]:
mcdf = pd.read_hdf(f, key="mcnu")

In [ ]:
mcdf

In [ ]:
df

In [ ]:
slc_is_reco = SlcInFV(df.slc.vertex) & (df.slc.tmatch.idx == 0)

In [ ]:
two_particles = (df.dist_to_vertex < 10).groupby(level=[0,1,2]).sum() >= 2

In [ ]:
two_contained_particles = ((df.dist_to_vertex < 10) & TrkInFV(df.pfp.trk.end)).groupby(level=[0,1,2]).sum() >= 2

In [ ]:
muid = (df.dist_to_vertex < 10) & TrkInFV(df.pfp.trk.end) & (df.pfp.trk.len > 10) &\
    (df.pfp.trk.chi2pid.I2.chi2_proton > 80) & (df.pfp.trk.chi2pid.I2.chi2_muon < 30)

two_muids = muid.groupby(level=[0,1,2]).sum() >= 2

In [ ]:
two_tracks = ((df.dist_to_vertex < 10) & (df.pfp.trackScore > 0.5)).groupby(level=[0,1,2]).sum() >= 2

In [ ]:
good_reco = (dmagdf(df.pfp.trk.start, df.pfp.trk.truth.p.start) < 10) &\
    (dmagdf(df.pfp.trk.end, df.pfp.trk.truth.p.end) < 10)

In [ ]:
good_reco1 = good_reco & (df.pfp.trk.truth.p.G4ID == 1)
good_reco2 = good_reco & (df.pfp.trk.truth.p.G4ID == 2)

good_reco = good_reco1.groupby(level=[0,1,2]).any() & good_reco2.groupby(level=[0,1,2]).any()

In [ ]:
cuts = [
    slc_is_reco,
    two_contained_particles,
    two_muids,
    two_tracks,
]

cutnames = [
    "Slice Reconstructed",
    "Two PFParticles < 10 cm to Vertex",
    "Two Muons < 10 cm to Vertex",
    "Two Tracks < 10 cm to Vertex",
]

In [ ]:
bins = np.linspace(0, 5, 11)
when_all = SlcInFV(mcdf.position) & (mcdf.npi == 0) & (mcdf.npi0 == 0) &\
    TrkInFV(mcdf.p0.end) & TrkInFV(mcdf.p1.end)

when_cut = when_all
for c, name in zip(cuts, cutnames):
    when_cut = when_cut & (c).groupby(level=[0, 1]).any()

    D, _ = np.histogram(mcdf[when_all].E, bins=bins)
    N, _ = np.histogram(mcdf[when_cut].E, bins=bins)  

    centers = (bins[1:] + bins[:-1])/2.

    plt.plot(centers, N/D, label=name)
    
    
plt.legend()


In [ ]:
cuts = [
    good_reco,
    two_contained_particles,
    two_muids,
#     two_tracks,
]

cutnames = [
    "Good Reconstruction",
    "Two PFParticles < 10 cm to Vertex",
    "Two Muon IDs < 10 cm to Vertex",
#     "Two Tracks < 10 cm to Vertex",
]

In [ ]:
bins = np.linspace(0, 5, 11)
when_all = SlcInFV(mcdf.position) & (mcdf.npi == 0) & (mcdf.npi0 == 0) &\
    TrkInFV(mcdf.p0.end) & TrkInFV(mcdf.p1.end)

when_cut = when_all
for c, name in zip(cuts, cutnames):
    when_cut = when_cut & (c).groupby(level=[0, 1]).any()

    D, _ = np.histogram(mcdf[when_all].E, bins=bins)
    N, _ = np.histogram(mcdf[when_cut].E, bins=bins)  

    centers = (bins[1:] + bins[:-1])/2.

    plt.plot(centers, N/D, label=name)
    
plt.legend()

plt.xlabel("Scalar Energy [GeV]")
plt.ylabel("Selection Fraction")
plt.title("Scalar Portal, $M_S = %i$ MeV" % MASS)

if dosave: plt.savefig(savedir + "scalar_good_reco_M%i_eff.pdf" % MASS)

In [ ]:
interesting_mcdf = when_all & two_particles.groupby(level=[0,1]).any() & ~two_tracks.groupby(level=[0,1]).any()

In [ ]:
mcdf[interesting_mcdf].position

In [ ]:
good_scalar = (mcdf.npi == 0) & (mcdf.npi0 == 0) &\
    TrkInFV(mcdf.p0.end) & TrkInFV(mcdf.p1.end)

In [ ]:
interesting_slcdf = df.good_scalar & good_reco & ~two_muids & (df.slc.tmatch.idx == 0) & (df.slc.truth.E > 0.5) 

In [ ]:
df["true_contained"] = TrkInFV(df.pfp.trk.truth.p.end)
df["reco_contained"] = TrkInFV(df.pfp.trk.end)

In [ ]:
cols = [
    ("slc", "truth", "E", "", "", ""),
    ("dist_to_vertex", "", "", "", "", ""),
    ("pfp", "trk", "len", "", "", ""),
    ("pfp", "trackScore", "", "", "", ""),
    ("pfp", "trk", "chi2pid", "I2", "chi2_proton", ""),
    ("pfp", "trk", "chi2pid", "I2", "chi2_muon", ""),
#     ("pfp", "trk", "end", "x", "", ""),
#     ("pfp", "trk", "end", "y", "", ""),
#     ("pfp", "trk", "end", "z", "", ""),
#     ("pfp", "trk", "truth", "p", "end", "x"),
#     ("pfp", "trk", "truth", "p", "end", "y"),
#     ("pfp", "trk", "truth", "p", "end", "z"),
    ("true_contained", "", "", "", "", ""),
    ("reco_contained", "", "", "", "", ""),
    ("pfp", "trk", "truth", "p", "pdg", ""),
    ("pfp", "trk", "truth", "p", "length", ""),
    ("pfp", "trk", "truth", "p", "G4ID", ""),
    
]

In [ ]:
df.loc[interesting_slcdf, cols][:50]

In [ ]:
muid.xs((20,15,2))